<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a585d17769c007a37972d8f49568071cd8eb877c749ae0dd7ac33ab433d0a309
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-07 14:12:10
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.42 C
-------------------
Today PnL: -57.69 K (-0.4%)
Current PnL: -22.09 L (-14.47%)
CY Booked + Current PnL: -9.44 L (-6.18%)
-------------------
Total profit:  1.92 L
Total loss:  -24.00 L
-------------------
Total Booked + Current PnL: 17.54 L (13.97%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.88%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 89.07 L (62.52%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.36%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-7.66,61.0,X-LC,6.42,230896.0,20530.0,8243.0,-1.07,9.76,3.57,13.68,37.0,2.49,1.63,27.89,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-3.88,67.0,M-LC,11.81,173768.0,32928.0,11556.0,1.79,23.38,6.65,31.59,66.0,2.85,1.23,56.06,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,103.20,53.0,M-SC,1.94,87521.0,-13256.0,13347.0,0.72,-13.15,15.25,0.09,245.0,-0.99,0.62,14.98,OX40N,NTT,DURABLES
33,ICICIGI,2252.93,-13.14,59.0,X-MC,5.87,149428.0,13295.0,17289.0,0.91,9.77,11.57,22.47,91.0,0.77,1.06,24.01,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-12.47,33.0,X-MC,2.89,86565.0,-4113.0,19018.0,-2.04,-4.54,21.97,16.44,101.0,-0.22,0.61,14.72,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,3033.0,19.39,64.0,X-SC,6.04,211240.0,20234.0,37474.0,3.60,10.59,17.74,30.21,157.0,0.54,1.49,33.69,X40N,NTT,FINANCE
84,VOLTAS,1530.0,-5.41,37.0,X-MC,4.41,198555.0,6813.0,30955.0,2.42,3.55,15.59,19.69,99.0,0.22,1.40,10.59,XY25,NTT,AC
79,UNIONBANK,163.0,-3.88,67.0,M-LC,11.81,173768.0,32928.0,11556.0,1.79,23.38,6.65,31.59,66.0,2.85,1.23,56.06,XY24,NTT,BANKS
34,ICICIPRULI,790.0,-19.50,59.0,X-MC,1.23,138414.0,2626.0,40126.0,1.44,1.93,28.99,31.48,107.0,0.07,0.98,14.29,X40,ATH,INSURANCE
82,VALIANTORG,838.0,-284.65,42.0,H-SC,11.28,102663.0,-60942.0,182258.0,1.31,-37.25,177.53,74.15,139.0,-0.33,0.73,31.60,XR,NTT,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.0,-42.49,35.0,H-LC,5.10,240288.0,-21331.0,140544.0,-4.25,-8.15,58.49,45.57,7.0,-0.15,1.70,2.24,AR,NTT,ELECTRICAL
71,SYMPHONY,1306.0,-44.56,44.0,M-SC,16.71,118162.0,-52979.0,52925.0,-3.91,-30.96,44.79,-0.03,196.0,-1.00,0.84,3.02,OX40N,NTT,DURABLES
70,SURYODAY,240.0,60.09,46.0,H-SC,14.96,146758.0,-32313.0,98049.0,-3.53,-18.04,66.81,36.71,135.0,-0.33,1.04,45.61,XR,NTT,BANKS
17,CERA,9475.0,-24.34,41.0,H-SC,1.85,137736.0,-38167.0,80190.0,-3.17,-21.70,58.22,23.89,149.0,-0.48,0.97,18.95,OX40N,NTT,CERAMICS
67,SIS,528.0,1988.60,41.0,H-SC,2.54,84290.0,-26742.0,50346.0,-2.61,-24.09,59.73,21.26,156.0,-0.53,0.60,13.88,OX40N,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,5.57,50.0,H-SC,7.93,125698.0,401.0,58525.0,-2.34,0.32,46.56,47.03,141.0,0.01,0.89,57.38,MH,ATH,POWER
56,RECLTD,446.00,42.86,46.0,M-LC,4.72,201878.0,-1292.0,43424.0,1.20,-0.64,21.51,20.74,55.0,-0.03,1.43,6.18,XY25,NTT,FINANCE
31,HINDZINC,730.22,25.96,46.0,M-LC,9.78,205494.0,418.0,112159.0,1.11,0.20,54.58,54.89,52.0,0.00,1.45,24.35,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,103.20,53.0,M-SC,1.94,87521.0,-13256.0,13347.0,0.72,-13.15,15.25,0.09,245.0,-0.99,0.62,14.98,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,44.0,H-SC,2.75,221400.0,-48267.0,84597.0,-0.88,-17.90,38.21,13.47,138.0,-0.57,1.57,12.58,XY24,NTT,PAINTS
17,CERA,9475.0,-24.34,41.0,H-SC,1.85,137736.0,-38167.0,80190.0,-3.17,-21.70,58.22,23.89,149.0,-0.48,0.97,18.95,OX40N,NTT,CERAMICS
42,JCHAC,2282.0,19921.74,62.0,M-SC,1.51,97882.0,-29923.0,29913.0,-0.01,-23.41,30.56,-0.01,232.0,-1.00,0.69,10.84,OX40N,NTT,AC
67,SIS,528.0,1988.60,41.0,H-SC,2.54,84290.0,-26742.0,50346.0,-2.61,-24.09,59.73,21.26,156.0,-0.53,0.60,13.88,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-3.88,67.0,M-LC,11.81,173768.0,32928.0,11556.0,1.79,23.38,6.65,31.59,66.0,2.85,1.23,56.06,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,50.76,60.0,M-MC,10.23,237481.0,12519.0,155146.0,-0.20,5.56,65.33,74.53,192.0,0.08,1.68,39.61,XY24,BTT,STEEL
31,HINDZINC,730.22,25.96,46.0,M-LC,9.78,205494.0,418.0,112159.0,1.11,0.20,54.58,54.89,52.0,0.00,1.45,24.35,X5K,ATH,METALS
46,KPIGREEN,731.05,5.57,50.0,H-SC,7.93,125698.0,401.0,58525.0,-2.34,0.32,46.56,47.03,141.0,0.01,0.89,57.38,MH,ATH,POWER
79,UNIONBANK,163.00,-3.88,67.0,M-LC,11.81,173768.0,32928.0,11556.0,1.79,23.38,6.65,31.59,66.0,2.85,1.23,56.06,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-40.54,52.0,M-SC,6.23,123179.0,8381.0,85659.0,-0.84,7.30,69.54,81.92,223.0,0.10,0.87,45.82,XR,NTT,DURABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-40.54,52.0,M-SC,6.23,123179.0,8381.0,85659.0,-0.84,7.30,69.54,81.92,223.0,0.10,0.87,45.82,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.71,51.0,H-SC,5.40,129341.0,6005.0,125616.0,-1.32,4.87,97.12,106.72,119.0,0.05,0.91,29.39,AR,ATH,MISC
46,KPIGREEN,731.05,5.57,50.0,H-SC,7.93,125698.0,401.0,58525.0,-2.34,0.32,46.56,47.03,141.0,0.01,0.89,57.38,MH,ATH,POWER
31,HINDZINC,730.22,25.96,46.0,M-LC,9.78,205494.0,418.0,112159.0,1.11,0.20,54.58,54.89,52.0,0.00,1.45,24.35,X5K,ATH,METALS
86,WIPRO,420.00,-16.52,38.0,M-LC,5.43,146661.0,-4284.0,113736.0,-1.46,-2.84,77.55,72.51,53.0,-0.04,1.04,3.01,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,22.0,X-LC,32.34,148050.0,-128473.0,238553.0,0.33,-46.46,161.13,39.81,54.0,-0.54,1.05,4.35,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,7.77,87291.0,-41379.0,86680.0,0.10,-32.16,99.30,35.20,219.0,-0.48,0.62,0.19,X40,NTT,FOOTWEAR
44,JSWINFRA,342.00,-24.57,26.0,X-MC,4.54,189472.0,-10875.0,41381.0,-1.13,-5.43,21.84,15.23,178.0,-0.26,1.34,25.07,X40N,NTT,REALTY
23,DMART,5391.45,-19.92,29.0,X-LC,7.94,204110.0,-6297.0,65458.0,-2.40,-2.99,32.07,28.12,38.0,-0.10,1.44,19.93,X40N,ATH,FMCG
30,HINDUNILVR,2922.00,-14.10,32.0,X-LC,6.41,241120.0,-15943.0,51069.0,-0.24,-6.20,21.18,13.67,24.0,-0.31,1.70,13.83,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-31.74,33.0,X-MC,0.18,197580.0,-3240.0,71030.0,0.08,-1.61,35.95,33.76,80.0,-0.05,1.40,3.50,X40,ATH,FMCG
41,ITC,452.00,-38.77,42.0,X-LC,1.08,197580.0,-2558.0,23453.0,-0.85,-1.28,11.87,10.44,4.0,-0.11,1.40,4.21,X40,NTT,FMCG
66,SIEMENS,4671.50,-5.18,38.0,H-LC,1.39,152310.0,-33785.0,81257.0,-1.21,-18.15,53.35,25.51,15.0,-0.42,1.08,12.24,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-8.87,35.0,X-MC,3.22,303954.0,-24037.0,130579.0,0.35,-7.33,42.96,32.48,92.0,-0.18,2.15,3.26,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-7.09,62.0,X-LC,4.13,243638.0,9494.0,39128.0,-2.37,4.05,16.06,20.77,86.0,0.24,1.72,11.25,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,7.77,87291.0,-41379.0,86680.0,0.10,-32.16,99.30,35.20,219.0,-0.48,0.62,0.19,X40,NTT,FOOTWEAR
51,PAGEIND,51769.93,-29.85,32.0,X-MC,7.42,79300.0,-4210.0,24242.0,-0.28,-5.04,30.57,23.98,82.0,-0.17,0.56,0.48,X40,ATH,APPARELS
53,QUESS,986.00,-50.41,35.0,X-SC,37.40,49474.0,-15532.0,166460.0,1.25,-23.89,336.46,232.18,198.0,-0.09,0.35,1.25,XY24,NTT,MISC
20,COLPAL,3726.84,-7.40,39.0,X-MC,7.29,216350.0,-47015.0,156335.0,-0.46,-17.85,72.26,41.51,84.0,-0.30,1.53,1.53,XY25,ATH,FMCG
1,ABB,7934.00,-42.49,35.0,H-LC,5.10,240288.0,-21331.0,140544.0,-4.25,-8.15,58.49,45.57,7.0,-0.15,1.70,2.24,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.41,35.0,X-SC,37.40,49474.0,-15532.0,166460.0,1.25,-23.89,336.46,232.18,198.0,-0.09,0.35,1.25,XY24,NTT,MISC
51,PAGEIND,51769.93,-29.85,32.0,X-MC,7.42,79300.0,-4210.0,24242.0,-0.28,-5.04,30.57,23.98,82.0,-0.17,0.56,0.48,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.27,42.0,X-SC,4.21,81385.0,-63775.0,139705.0,-1.03,-43.93,171.66,52.31,136.0,-0.46,0.58,8.94,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-12.47,33.0,X-MC,2.89,86565.0,-4113.0,19018.0,-2.04,-4.54,21.97,16.44,101.0,-0.22,0.61,14.72,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,7.77,87291.0,-41379.0,86680.0,0.10,-32.16,99.30,35.20,219.0,-0.48,0.62,0.19,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-29.97,44.0,X-LC,9.73,279631.0,-66325.0,133020.0,-1.20,-19.17,47.57,19.28,1.0,-0.50,1.98,3.38,X40,ATH,IT
39,INFY,2275.00,-19.24,50.0,X-LC,4.98,314047.0,1159.0,170528.0,0.52,0.37,54.30,54.87,3.0,0.01,2.22,8.68,X40,BTT,IT
41,ITC,452.00,-38.77,42.0,X-LC,1.08,197580.0,-2558.0,23453.0,-0.85,-1.28,11.87,10.44,4.0,-0.11,1.40,4.21,X40,NTT,FMCG
83,VBL,671.64,-16.88,52.0,X-LC,6.34,299286.0,-16556.0,129232.0,-0.58,-5.24,43.18,35.67,5.0,-0.13,2.12,7.89,X40N,ATH,FMCG
1,ABB,7934.00,-42.49,35.0,H-LC,5.10,240288.0,-21331.0,140544.0,-4.25,-8.15,58.49,45.57,7.0,-0.15,1.70,2.24,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-49.70,33.0,L-SC,27.48,73440.0,-40109.0,80160.0,-2.45,-35.32,109.15,35.27,268.0,-0.50,0.52,79.75,XR,NTT,HOTELS
7,ASIANTILES,137.00,7316.67,61.0,L-SC,12.27,81370.0,-12440.0,89059.0,-2.46,-13.26,109.45,81.67,269.0,-0.14,0.58,57.58,XR,NTT,CERAMICS
49,MASFIN,398.61,-19.15,49.0,H-SC,8.34,91980.0,-6000.0,27603.0,0.79,-6.12,30.01,22.05,152.0,-0.22,0.65,33.70,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-40.54,52.0,M-SC,6.23,123179.0,8381.0,85659.0,-0.84,7.30,69.54,81.92,223.0,0.10,0.87,45.82,XR,NTT,DURABLES
46,KPIGREEN,731.05,5.57,50.0,H-SC,7.93,125698.0,401.0,58525.0,-2.34,0.32,46.56,47.03,141.0,0.01,0.89,57.38,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,73.00,64.0,H-SC,3.99,150109.0,-32666.0,143729.0,0.08,-17.87,95.75,60.77,125.0,-0.23,1.06,35.18,XR,NTT,JEWELLERY
68,SONACOMS,806.63,-31.27,69.0,M-SC,6.44,85505.0,-15654.0,55655.0,1.18,-15.47,65.09,39.54,202.0,-0.28,0.60,20.57,AR,ATH,AUTO
46,KPIGREEN,731.05,5.57,50.0,H-SC,7.93,125698.0,401.0,58525.0,-2.34,0.32,46.56,47.03,141.0,0.01,0.89,57.38,MH,ATH,POWER
62,SAMMAANCAP,326.00,-197.73,70.0,M-SC,34.06,168858.0,19638.0,124550.0,1.25,13.16,73.76,96.62,208.0,0.16,1.19,82.74,XY25,NTT,FINANCE
5,ANGELONE,3033.00,19.39,64.0,X-SC,6.04,211240.0,20234.0,37474.0,3.60,10.59,17.74,30.21,157.0,0.54,1.49,33.69,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.72
1,25,44.32
2,50,75.50


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.81
LC    33.14
MC    23.07
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.97
X40      16.28
X40N     12.48
XY25     11.94
XR        9.88
AR        7.79
OX40N     6.53
X200      1.77
X5K       1.45
SR        1.04
MH        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.22
X-LC    22.46
X-MC    18.74
M-SC    12.59
X-SC     5.56
M-LC     5.15
H-LC     4.55
H-MC     2.33
M-MC     1.68
L-SC     1.44
L-LC     0.98
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.28,-4.72,38.57
IT,12.45,-19.49,83.50
FINANCE,12.21,-9.44,59.05
MISC,7.78,-18.03,77.56
ELECTRICAL,5.70,-12.28,52.08
PAINTS,5.60,-16.77,34.18
BANKS,4.05,-18.20,86.96
INSURANCE,3.91,-0.14,37.48
AUTO,2.73,-48.82,110.93


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3336536.0,22
XR,1260402.0,13
AR,1185447.0,9
X40,856176.0,12
X40N,703783.0,9
OX40N,563156.0,9
XY25,482743.0,8
SR,274348.0,2
X5K,112159.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3406469.0,24
M-SC,1262670.0,15
X-MC,1224766.0,14
X-LC,1101898.0,13
X-SC,563239.0,6
H-LC,295648.0,3
M-LC,280875.0,4
L-SC,259091.0,3
H-MC,259083.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1196759.0      6
           AR         840503.0      5
           XR         785242.0      7
M-SC       XY24       733500.0      6
X-MC       XY24       575326.0      4
X-LC       X40        467212.0      5
           XY24       314113.0      2
X-SC       X40N       310099.0      4
X-MC       X40        302284.0      6
H-SC       SR         274348.0      2
           OX40N      251092.0      3
X-LC       X40N       233818.0      3
H-LC       AR         221801.0      2
X-MC       XY25       187290.0      2
H-MC       XY24       183676.0      1
L-SC       XR         169219.0      2
X-SC       XY24       166460.0      1
X-MC       X40N       159866.0      2
M-MC       XY24       155146.0      1
M-SC       OX40N      146785.0      4
           XR         134692.0      2
           XY25       124550.0      1
           AR         123143.0      2
M-LC       XR         113736.0      1
           X5K        112159.0      1
L-SC       OX40N       89872.0      1
X-LC       XY25        86755.0      3
X-SC       X40         86680.0      1
H-MC       OX40N       75407.0      1
H-LC       X200        73847.0      1
H-SC       MH          58525.0      1
L-MC       XR          57513.0      1
M-LC       XY25        43424.0      1
L-LC       XY25        40724.0      1
M-LC       XY24        11556.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
